<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#导入相关的包" data-toc-modified-id="导入相关的包-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入相关的包</a></span></li><li><span><a href="#导入相关数据" data-toc-modified-id="导入相关数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入相关数据</a></span></li><li><span><a href="#形成训练数据" data-toc-modified-id="形成训练数据-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>形成训练数据</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>训练模型</a></span></li></ul></div>

# 导入相关的包

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

D:\ProgramAPP\python\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 导入相关数据

In [15]:
%%time
trains = pd.read_csv('../raw_data/d_train.csv',encoding="gbk")
tests = pd.read_csv("../raw_data/d_test_A.csv",encoding="gbk")
trains.drop(trains[trains["年龄"] >= 84].index,inplace=True)
fea_train = pd.read_csv("../raw_data/fea_train.csv")
fea_test = pd.read_csv("../raw_data/fea_test.csv")
fea_train1 = pd.read_csv("../raw_data/fea_train_1.csv")
fea_test1 = pd.read_csv("../raw_data/fea_test_1.csv")
fea_train2 = pd.read_csv("../raw_data/fea_train_2.csv")
fea_test2 = pd.read_csv("../raw_data/fea_test_2.csv")
trains = pd.merge(trains, fea_train2, how="left",on="id")
trains = pd.merge(trains, fea_train, how="left",on="id")
# trains = pd.merge(trains, fea_train1, how="left",on="id")
tests = pd.merge(tests, fea_test, how="left",on="id")
tests = pd.merge(tests, fea_test1, how="left",on="id")
# tests = pd.merge(tests, fea_test2, how="left",on="id")
trains["血糖"] = trains["血糖"].apply(lambda x: 1 if x < 4.5 else 0)
trains["血糖"] = trains["血糖"].astype(np.int32)

Wall time: 543 ms


In [16]:
sum(trains["血糖"])

307

# 形成训练数据

In [17]:
%%time
def make_feat(train,test):
    train_id = train.id.values.copy()
    test_id = test.id.values.copy()
    data = pd.concat([train,test])
    data['性别'] = data['性别'].map({'男': 1,'女': 0})
    # data["性别"] = data['性别'].astype(int)
    # data['date'] = (pd.to_datetime(data['date']) - parse('2017-10-09')).dt.days
    # data['体检日期'] = (pd.to_datetime(data['体检日期']) - parse('2017-10-09')).dt.days
    data.drop("体检日期",axis = 1,inplace= True)
    # data.fillna(value = -1, inplace = True)
    train_feat = data[data.id.isin(train_id)]
    test_feat = data[data.id.isin(test_id)]
    return train_feat,test_feat
train, test = make_feat(trains, tests)
predictors = [f for f in list(train.columns) if f not in ["血糖","blood_sugar","id","blood_sugar_log","体检日期"]]
# X_train, X_test, y_train,y_test = train_test_split(train[predictors], train["血糖"],test_size=0.1,random_state=42)
xgb_train = xgb.DMatrix(train[predictors], label=train["血糖"])
# xgb_eval = xgb.DMatrix(X_test, label=y_test)
xgb_test = xgb.DMatrix(test[predictors])

Wall time: 239 ms


In [5]:
train.shape
test.shape

(1000, 103)

# 训练模型

In [18]:
params={'booster':'gbtree',
    'objective': 'binary:logistic',
    'scale_pos_weight':float(len(train["血糖"]) - sum(train["血糖"])) / sum(train["血糖"]),
    'eval_metric': 'auc',
    "subsample":0.65,
    "colsample_bytree":0.8,
    "lambda":5,
    "seed":1024,
    "silent":1,
    "verbose":0,
    "max_depth":7,
    "alpha":0.01,
    "gamma":0.6
    }

In [19]:
%%time
watchlist  = [(xgb_train,'train')]
#通过cv找最佳的nround
cv_log = xgb.cv(params,xgb_train,num_boost_round=25000,nfold=5,metrics='auc',early_stopping_rounds=50,seed=1024)

Wall time: 14.4 s


In [20]:
cv_log

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.589759,0.039457,0.785767,8.909739e-03
1,0.584315,0.032787,0.856076,7.935979e-03
2,0.583758,0.043888,0.896485,7.335993e-03
3,0.604961,0.042227,0.924224,3.749455e-03
4,0.598711,0.034205,0.938825,2.403528e-03
5,0.603907,0.034750,0.951963,2.858483e-03
6,0.604072,0.038896,0.960967,1.807797e-03
7,0.603323,0.036303,0.970963,2.151622e-03
8,0.607401,0.037415,0.978226,2.491460e-03
9,0.612106,0.043000,0.983191,2.533647e-03


In [21]:
%%time
bst_auc= cv_log['test-auc-mean'].max()
cv_log['nb'] = cv_log.index
cv_log.index = cv_log['test-auc-mean']
bst_nb = cv_log.nb.to_dict()[bst_auc]
#train
watchlist  = [(xgb_train,'train')]
model = xgb.train(params,xgb_train,num_boost_round=bst_nb+50,evals=watchlist)

[0]	train-auc:0.778714
[1]	train-auc:0.837598
[2]	train-auc:0.87241
[3]	train-auc:0.896051
[4]	train-auc:0.92095
[5]	train-auc:0.933955
[6]	train-auc:0.950999
[7]	train-auc:0.956242
[8]	train-auc:0.965601
[9]	train-auc:0.970499
[10]	train-auc:0.975288
[11]	train-auc:0.978658
[12]	train-auc:0.982627
[13]	train-auc:0.986566
[14]	train-auc:0.989675
[15]	train-auc:0.99094
[16]	train-auc:0.992046
[17]	train-auc:0.99396
[18]	train-auc:0.995672
[19]	train-auc:0.997072
[20]	train-auc:0.997773
[21]	train-auc:0.998467
[22]	train-auc:0.998533
[23]	train-auc:0.999012
[24]	train-auc:0.999119
[25]	train-auc:0.999407
[26]	train-auc:0.999564
[27]	train-auc:0.999708
[28]	train-auc:0.99976
[29]	train-auc:0.999788
[30]	train-auc:0.999876
[31]	train-auc:0.999917
[32]	train-auc:0.999935
[33]	train-auc:0.999939
[34]	train-auc:0.999955
[35]	train-auc:0.999988
[36]	train-auc:0.999987
[37]	train-auc:0.999995
[38]	train-auc:0.999996
[39]	train-auc:0.999993
[40]	train-auc:0.999994
[41]	train-auc:0.999997
[42]	tr

In [22]:
test_pred = model.predict(xgb_test)

In [23]:

test_result = test
test_result["lt45_prob"] = test_pred
# test_result.to_csv("gt98_prob.csv",index=None,encoding='utf-8')
test_result[["id","lt45_prob"]].to_csv("../classification/result/lt45.csv",index=False)
# 

In [24]:
test_result[test_result.lt45_prob.values > 0.499]["lt45_prob"].count()

1